This file join co_session with plan_session.<br>
This file dont have to be executed, it exist only for explaining how i achieved the file "session.csv"
## Library

In [2]:
## Pandas for reading 
import pandas as pd
## For transforming millisecond to date
from datetime import datetime
## Clear Output
from IPython.display import clear_output
## For nan array
import numpy as np

## Reading

In [8]:
## Core_session dataset
core_session = pd.read_csv("../../data/core_protector/co_session.csv")
## pla_session dataset
server_session = pd.read_csv("../../data/server_table/plan_sessions.csv", index_col = "id")
## co_user dataset (id -> uuid)
co_user = pd.read_csv("../../data/core_protector/co_user.csv", index_col = "id")

## Time
### server_session
How much time the player had passes online 

In [9]:
date = []
for i in range(server_session.__len__()):
    ## Removing of the "millisecond" part
    date.append(pd.Timestamp(datetime.fromtimestamp((server_session.iloc[i].session_start/1000.0).astype(int))))

#### Adding/Removing Columns

In [10]:
## Adding "time" columns
server_session.insert(4, "time", server_session["session_end"] - server_session["session_start"])
## Modifing "session start" with a human time
server_session["session_start"] = date
## Removing "session_end" and "server_uuid"
server_session.drop(columns = ["session_end", "server_uuid"], inplace = True)

### core_session

In [11]:
date = []
for i in range(core_session.__len__()):
    ## Transforming ms time to human
    date.append(pd.Timestamp(datetime.fromtimestamp(core_session.iloc[i].time)))

#### Editing Columns

In [12]:
## Modifing "session start" with a human time
core_session["time"] = date
## Replacing id with uuid
core_session["user"] = co_user.loc[core_session["user"]].uuid.values
## Renaming column user to uuid 
core_session.rename(columns = {"user" : "uuid"}, inplace = True)
## Removing wid column
core_session.drop(columns = ["wid"], inplace = True)

### Adding coords join and coords left to our server_session

#### Removing duplicates
So, for some reason there are more people that has left the server then people that has join the server.<br>
So i have to remove every error that the server made.<br>

In [13]:
## Lenght of core_session
len_ = core_session.__len__()
## Max Distance
distance = 1000
## Final Output
coordinates = pd.DataFrame({"uuid" : [], "time" : [], "x_join" : [], "x_left" : [], "y_join" : [], "y_left" : [], "z_join" : [], "z_left" : []})
## Number Success
success = 0

In [14]:
## Since this will take long (my pc took 8 hours) i created a backup called "coordinates.pkl".
## If you still want to spend 8 hours, set "True" in the if that is under this line
if False:
    for i in range(len_):
        val = core_session.iloc[i]
        ## If the guy join
        if val.action == 1:
            ## Iterate
            for j in range(distance):
                ## If we are in the range
                if i + j < len_:
                    ## If the uuid is the same and it's a left
                    if val.uuid == core_session.iloc[i + j].uuid and core_session.iloc[i + j].action == 0:
                        ## Add and left
                        coordinates = coordinates.append({
                                            "uuid" : val.uuid, "time" : val.time, 
                                            "x_join" : val.x, "x_left" : core_session.iloc[i + j].x,
                                            "y_join" : val.y, "y_left" : core_session.iloc[i + j].y,
                                            "z_join" : val.z, "z_left" : core_session.iloc[i + j].z}, 
                                             ignore_index = True)
                        success += 1
                        break

        if i % 500 == 0:
            clear_output()
            print("Time: {}%\nSuccess: {}/{}".format(i / len_ * 100, success, i + 1))

    clear_output()
    print("Time: {}%\nSuccess: {}/{}".format(i / len_ * 100, success, i + 1))
else:
    coordinates = pd.read_pickle("./backup/coordinates.pkl")

### Adding to server_session
Now we have to join our two dataframe <br>
Since server_session and coordinates (for some reason) sometimes they dont have what the other has,<br>
we have to create an algorithm that find every elements that both of them have<br>
We use server_session as a base and we'll iterate coordinates <br>
Every new value of coordinates, we take his time, with this value we search if server session <br>
has in some row the same value and, if yes, we check in all of these rows if the uuid is the same. <br>
If yes we can finally add our coordinates

In [15]:
## Adding some empty columns
server_session = pd.concat([server_session,pd.DataFrame(columns=["x_join", "x_left", "y_join", "y_left", "z_join", "z_left", "id"])])

In [16]:
for i in range(coordinates.__len__()):
    ## Finding every possibilities
    possibilites = server_session["session_start"] == coordinates["time"][i]
    ## if it's != 0 (so we have 2 values), We have at least 1 possibily
    if possibilites.value_counts().__len__() != 1:
        ## Lets store all our possibilites
        possibilites = server_session[possibilites]
        ## Iterate
        for j in possibilites.index:
            ## If there is a match
            if possibilites.loc[j].uuid == coordinates.iloc[i].uuid:
                ## We can add our values
                server_session.loc[j, "x_join" : "z_left"] = coordinates.loc[i, "x_join" : "z_left"]
                server_session.loc[j, "id"] = i
                continue
    ## Print
    if i % 500 == 0:
        clear_output()
        print("Time: {}%".format(i / coordinates.__len__()* 100))

Time: 0.3179037006110109%



KeyboardInterrupt



### Add name to uuid

In [5]:
## Import uuid
uuid_db = pd.read_pickle("../../data/ufficial/pickle/uuid.pkl")

In [6]:
server_session["name"] = server_session.apply(lambda x : uuid_db[uuid_db["uuid"] == x["uuid"]].iloc[0].nickname,\
                                       axis=1)

### Exporting

In [10]:
server_session.to_csv("../../data/ufficial/csv/session.csv")
server_session.to_pickle("../../data/ufficial/pickle/session.pkl")

In [8]:
server_session

,uuid,session_start,time,mob_kills,deaths,afk_time,x_join,x_left,y_join,y_left,z_join,z_left,name
1,b751b9ef-2b6f-4678-9908-718bf93d7089,2020-02-07 22:11:54,25620.0,0.0,0.0,0.0,32,32,73,73,1,1,SalC1
2,b751b9ef-2b6f-4678-9908-718bf93d7089,2020-02-07 22:12:21,3690.0,0.0,0.0,0.0,32,32,73,73,1,1,SalC1
3,b751b9ef-2b6f-4678-9908-718bf93d7089,2020-02-07 22:51:01,3124.0,0.0,0.0,0.0,32,32,73,73,1,1,SalC1
4,f2b5fb77-1b63-422d-851f-c3bbd6b0c418,2020-02-07 23:43:34,5205.0,0.0,0.0,0.0,24,22,70,69,126,127,BoatMeat
5,6b1bcae8-d43a-41ee-a74e-0391fc8a3541,2020-02-07 23:44:03,278.0,0.0,0.0,0.0,-75,-75,85,85,45,45,8chan_
...,...,...,...,...,...,...,...,...,...,...,...,...,...
493199,01d133c2-d7ee-4808-906f-4601b9f285fe,2020-04-14 02:30:30,213146.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,spit4520
493200,6d026b92-5b15-4555-84fd-db4d1162eac9,2020-04-14 02:23:37,626550.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,CactusDuper
493201,13ed27d3-b345-4bac-aae4-497d8e30dd9b,2020-04-14 02:16:18,1065471.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,WittleRain
493202,c0ce433b-c1de-4535-a2e5-4a22e621759d,2020-04-14 02:16:21,1062235.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,mcrcortex
